In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time

article_name = []
article_date = []
article_words = []
article_len = []
article_URL = []
article_category = []

categories =['i_politica.php', 'i_sociedad.php', 'i_nacional.php']

for month in range(11,12):
    for day in range (1,5):
        for category in categories:
        
            #this defines the changing name of each article
            name = category

            if day > 9:
                URL = f'https://www.eldiario.net/noticias/2019/2019_{str(month)}/nt19{str(month)}{str(day)}/{name}'
                #print(URL)
            else:
                URL = f'https://www.eldiario.net/noticias/2019/2019_{str(month)}/nt19{str(month)}0{str(day)}/{name}'
                #print(URL)

            #------------------------------------------------
            #--------------- FIRST PART ---------------------
            #------------------------------------------------

            #This part of the code retrieves the links of the news (1 for now).

            #The first request
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, 'html.parser')

            #The part where the desired info is:
            main_body = soup.find(id='col_a')

            #print(main_body.prettify())

            #once we have the main body, this is the way to get the link.
            links_with_text=[]
            for a in main_body.find_all('a', href=True): 
                if a.text: 
                    links_with_text.append(a['href'])

            #Now, the 'name' will change and will be the link of the new we want to analyze.
            name = str(links_with_text[0])

            #------------------------------------------------
            #--------------- SECOND PART --------------------
            #------------------------------------------------

            #This part analyze the given new
            if day >= 10:
                URL = f'https://www.eldiario.net/noticias/2019/2019_{str(month)}/nt19{str(month)}{str(day)}/{name}'
                #print(URL)
            elif day < 10:
                URL = f'https://www.eldiario.net/noticias/2019/2019_{str(month)}/nt19{str(month)}0{str(day)}/{name}'
                #print(URL)      

            #This part analyze the given new
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, 'html.parser')

            # The part where the desired info is:
            main_body = soup.find(id='col_a')

            #print(results.prettify())

            #Accesing the main body of the article
            main_body_parts = []
            for part in main_body.select('div'): 
                #print('---------------------')
                #print(elem)
                main_body_parts.append(part)

            #print(divs[0])

            # Here is my beloved text
            main_text = main_body_parts[0].find('div', class_ ="nota_txt")
            texto_completo = main_text.find_all('p')

            #Lets get rid of everything but the text itsef.
            #It will be store in 'phrases'.
            phrases = []
            for fragmento in texto_completo:
                phrases.append(fragmento.getText())    

            #--------------------    

            # In this part of the code, the phrases are split and store
            # along with more info, such as date or number of words.
            list_words = []

            for phrase in phrases:
                list_words.append(phrase.split())

            #super fast way to flatten a nested list
            flatten_words = [x for y in list_words for x in y]

            #super fast way to remove empty lists
            only_words = [word for word in flatten_words if word != []]
            
            #writting nice the category
            if category == 'i_politica.php':
                article_category.append('politica')
            elif category == 'i_sociedad.php':
                article_category.append('sociedad')
            else:
                article_category.append('nacional')

            #time to add the info to different lists
            article_name.append(str(name))
            article_date.append(f'2019/{str(month)}/{str(day)}')
            article_category.append(category)
            article_words.append(only_words)
            article_len.append(len(only_words))
            article_URL.append(URL)

            time.sleep(1)
        
#------------------------------------------------
#--------------- PANDAS PART --------------------
#------------------------------------------------
                            
df = pd.DataFrame(list(zip(article_name, article_date,article_category,article_words,article_len,article_URL)), 
               columns =['article_name', 'article_date','article_category', 'article_words','article_len','article_URL'])
                            
df.to_csv('larazon.csv')

In [2]:
df

,article_name,article_date,article_category,article_words,article_len,article_URL
0,politica.php?n=45&-morales-pide-pausa-los-civi...,2019/11/1,politica,"[Después, de, dos, muertes, en, el, municipio,...",638,https://www.eldiario.net/noticias/2019/2019_11...
1,sociedad.php?n=15&-ministerio-publico-no-escla...,2019/11/1,i_politica.php,"[En, los, más, de, 13, años, de, gestión, de, ...",887,https://www.eldiario.net/noticias/2019/2019_11...
2,nacional.php?n=48&-afines-al-gobierno-agredier...,2019/11/1,sociedad,"[Al, mediodía, de, ayer, un, numeroso, grupo, ...",468,https://www.eldiario.net/noticias/2019/2019_11...
3,politica.php?n=53&-se-fractura-mision-de-oea-p...,2019/11/2,i_sociedad.php,"[La, misión, de, expertos, de, la, Organizació...",413,https://www.eldiario.net/noticias/2019/2019_11...
4,sociedad.php?n=19&-6-aprehendidos-por-muerte-d...,2019/11/2,nacional,"[El, fiscal, departamental, de, Santa, Cruz,, ...",192,https://www.eldiario.net/noticias/2019/2019_11...
5,nacional.php?n=16&-bloqueos-se-mantuvieron-en-...,2019/11/2,i_nacional.php,"[Pese, a, la, festividad, de, Todos, Santos,, ...",304,https://www.eldiario.net/noticias/2019/2019_11...
6,politica.php?n=20&-piden-cumbre-nacional-para-...,2019/11/3,politica,"[El, analista, Iván, Arias,, señaló, que, las,...",723,https://www.eldiario.net/noticias/2019/2019_11...
7,sociedad.php?n=27&-sujeto-de-la-escopeta-dio-p...,2019/11/3,i_politica.php,"[Wilson, Rodríguez, Peralta,, el, sujeto, que,...",373,https://www.eldiario.net/noticias/2019/2019_11...
8,nacional.php?n=46&-dinamitazos-y-violencia-des...,2019/11/3,sociedad,"[Detonaciones, de, dinamitas, causadas, por, m...",568,https://www.eldiario.net/noticias/2019/2019_11...
9,politica.php?n=77&-camacho-a-morales-vamos-a-s...,2019/11/4,i_sociedad.php,"[El, presidente, del, Comité, Cívico, Pro, San...",577,https://www.eldiario.net/noticias/2019/2019_11...
